In [ ]:
import sys as _sys
import os

currentPath = os.path.abspath(os.getcwd())

split = currentPath.split("Cshells")
if len(split)<2:
    print("Please rename the repository 'Cshells'")
    raise ValueError
pathToPythonScripts = split[0] + "Cshells/python/"
pathToCubicSplines  = split[0] + "Cshells/ext/torchcubicspline/"

_sys.path.insert(0, pathToPythonScripts)
_sys.path.insert(0, pathToCubicSplines)

In [ ]:
import MeshFEM
import ElasticRods

import average_angle_linkages
from bending_validation import suppress_stdout as so
import elastic_rods
import json
from linkage_vis import LinkageViewer
import math
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import CubicSpline
from scipy.interpolate import interp1d
import torch

from CShell import CShell, GetEdgesFromCurves
from CShellToJSON import ExtractQuantitiesPerSegment
from LinkageTopologies import RegularTopology
from open_average_angle_linkage import open_average_angle_linkage
import py_newton_optimizer

torch.set_default_dtype(torch.float64)
    
def ToNumpy(tensor):
    return tensor.cpu().detach().clone().numpy()

PI = math.pi

# Define a linkage

In [ ]:
nJa  = 3 # Number of joints for the first family of joints
nJb  = 2  # Number of joints for the second family of joints

nJ, curves, curvesFamily = RegularTopology(nJa, nJb)

lengthX   = 10.0
lengthY   = 7.0

xPos = torch.tile(torch.linspace(0, lengthX, nJa), (nJb,))
yPos = torch.repeat_interleave(torch.linspace(0, lengthY, nJb), nJa, dim=0)

xOffset = 0.5
yOffset = 0.8
xPos   += xOffset * torch.tensor([0.0, -1.0, 0.0, 0.0, 1.0, 0.0])
yPos   += yOffset * torch.tensor([1.0, 0.0, 1.0, -1.0, 0.0, -1.0])

joints = torch.zeros(size=(nJ, 3))
joints[:, 0] = xPos
joints[:, 1] = yPos

rodEdges, rodEdgeToCurve = GetEdgesFromCurves(curves)

In [ ]:
jointsPosition = ToNumpy(joints)
rodMaterial    = elastic_rods.RodMaterial('rectangle', 2000, 0.3, [0.60, 0.30])

# For the discretization
subdivision = 10

flatLinkageComp = elastic_rods.RodLinkage(jointsPosition, rodEdges,
                                          rod_interleaving_type=elastic_rods.InterleavingType.xshell, subdivision=subdivision)
flatLinkageComp = average_angle_linkages.AverageAngleLinkage(flatLinkageComp)

driver = flatLinkageComp.centralJoint()
flatLinkageComp.setMaterial(rodMaterial)
flatLinkageComp.set_design_parameter_config(True, True, True) # Keep length, remove rest curvature, and update the design parameters

jdo       = flatLinkageComp.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
with so(): elastic_rods.compute_equilibrium(flatLinkageComp, fixedVars=fixedVars)

flatViewComp = LinkageViewer(flatLinkageComp, width=1024, height=640)

In [ ]:
flatViewComp.show()

In [ ]:
deployedLinkageComp = average_angle_linkages.AverageAngleLinkage(flatLinkageComp)
deployedViewComp    = LinkageViewer(deployedLinkageComp, width=1024, height=640)

def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.gradTol = 1.0e-4
    return average_angle_linkages.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)

deployedViewComp.show()

In [ ]:
alphaTar        = np.deg2rad(155)
numOpeningSteps = 40
maxNewtonIterIntermediate = 20

with so(): open_average_angle_linkage(deployedLinkageComp, driver, alphaTar - deployedLinkageComp.averageJointAngle, numOpeningSteps, 
                                      deployedViewComp, equilibriumSolver=equilibriumSolver, 
                                      maxNewtonIterationsIntermediate=maxNewtonIterIntermediate)

# Define a C-shell

In [ ]:
mult = 5
subdivision = 10

nCPperRodEdge = [(len(crv) - 1) * [1] for crv in curves]
totNCP = sum([sum(el) for el in nCPperRodEdge])
curvesDoF = torch.zeros(size=(2 * nJ + totNCP,))
curvesDoF[:2*nJ] = torch.tensor(flatLinkageComp.jointPositions().reshape(-1, 3)[:, :2].reshape(-1,))
curvesDoF[2*nJ] = -0.15
curvesDoF[2*nJ+1] = -0.15
curvesDoF[2*nJ+2] = 0.15
curvesDoF[2*nJ+3] = 0.15

cshell = CShell(
    curvesDoF, nJ, curves, curvesFamily, nCPperRodEdge, alphaTar, mult, subdivision, 
    rodMaterial=rodMaterial, pathSurf=None, newtonOptimizerOptions=None, optimizeAlpha=True, 
    useSAL=False, attractionMesh=None, targetMesh=None,
    dictWeights=None, linkagesGuess=None,
    numOpeningSteps=40, maxNewtonIterIntermediate=20, flatOnly=False
)

deltaAlpha = cshell.flatLinkage.averageJointAngle - flatLinkageComp.averageJointAngle
deltaAlpha = 0.3

alphaTarCshell = cshell.alphaTar + 0.1

cshell = CShell(
    curvesDoF, nJ, curves, curvesFamily, nCPperRodEdge, alphaTarCshell, mult, subdivision, 
    rodMaterial=rodMaterial, pathSurf=None, newtonOptimizerOptions=None, optimizeAlpha=True, 
    useSAL=False, attractionMesh=None, targetMesh=None,
    dictWeights=None, linkagesGuess=None,
    numOpeningSteps=40, maxNewtonIterIntermediate=20, flatOnly=False
)

In [ ]:
cshell.PlotCurveLinkage()

In [ ]:
cshell.deployedView.show()